In [ ]:
# From a really helpful article on encoder selection:
# https://towardsdatascience.com/an-easier-way-to-encode-categorical-features-d840ff6b3900

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
import category_encoders as ce

In [ ]:
X = df4models.drop('reordered', axis=1)
y = df4models['reordered']

numeric_features = list(df4models.select_dtypes(include=[
    'int64', 'float64']).drop(['reordered'], axis=1).columns)

categorical_features = list(df4models.select_dtypes(include=[
    'object']).columns)

le = preprocessing.LabelEncoder()
label_encoder = le.fit(y)
y = label_encoder.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2)

In [ ]:
encoder_list = [ce.backward_difference.BackwardDifferenceEncoder, 
               ce.basen.BaseNEncoder,
               ce.binary.BinaryEncoder,
                ce.cat_boost.CatBoostEncoder,
                ce.hashing.HashingEncoder,
                ce.helmert.HelmertEncoder,
                ce.james_stein.JamesSteinEncoder,
                ce.one_hot.OneHotEncoder,
                ce.leave_one_out.LeaveOneOutEncoder,
                ce.m_estimate.MEstimateEncoder,
                ce.ordinal.OrdinalEncoder,
                ce.polynomial.PolynomialEncoder,
                ce.sum_coding.SumEncoder,
                ce.target_encoder.TargetEncoder,
                ce.woe.WOEEncoder]

for encoder in encoder_list:
    
    numeric_transformer = Pipeline(steps=[('scaler', 
                                            StandardScaler())])
    categorical_transformer = Pipeline(steps=[('woe', encoder())])
    
    preprocessor = ColumnTransformer(transformers=[(
        'num', numeric_transformer, numeric_features),(
        'cat', categorical_transformer, categorical_features)])
    
    pipe = Pipeline(steps=[('preprocessor', preprocessor),(
        'classifier', RandomForestClassifier(n_estimators=500))])
    
    model = pipe.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    print(encoder)
    print(f1_score(y_test, y_pred, average='macro'))